# Downloading necessary libraries

In [1]:
!pip3 -qq install torch torchvision torchaudio
!pip -qq install matplotlib pandas numpy scikit-learn openpyxl

# Import necessary libraries

In [2]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Device agnostic

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load the data

In [4]:
data = pd.read_excel("data/openings.xlsx")
data.head()

,space,room_size,no_of_users,no_of_openings,width_1,width_2,width_3,width_4,width_5
0,Area for standing,50,40,1,0.75,0.00,0.00,0.00,0.00
1,Area for standing,75,200,2,0.75,0.75,0.00,0.00,0.00
2,"assembly hall,concourse",100,300,3,0.75,0.75,0.75,0.00,0.00
3,check in que,125,400,4,0.75,0.75,0.75,0.75,0.00
4,Office,150,500,5,0.75,0.75,0.75,0.75,0.75


In [5]:
X = data.drop(['no_of_openings', 'width_1', 'width_2', 'width_3', 'width_4', 'width_5'], axis=1)

In [6]:
y = data.drop(["space", "room_size", "no_of_users"], axis=1)

## Split the data into train and test data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Data Preprocessing

In [8]:
num_attribs = ["room_size", "no_of_users"]
cat_attribs = ["space"]

num_pipeline = Pipeline([('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [9]:
X_train_tensor = torch.Tensor(full_pipeline.fit_transform(X_train)).to(device)
X_test_tensor = torch.Tensor(full_pipeline.transform(X_test)).to(device)
y_train_tensor = torch.from_numpy(y_train.values).float().to(device)
y_test_tensor = torch.from_numpy(y_test.values).float().to(device)

# Building the model